## Finetune Model with ESPnet-Easy

In this notebook, we will explore the process of finetuning a pretrained model using the Librispeech-100 dataset. We'll start by downloading a pretrained model from the Hugging Face model hub and apply Low-Rank Adaptation (LoRA) techniques to reduce the number of training parameters.

In this notebook, we assume that the dump files have been already created. If you need guidance on creating the dump files, you can refer to the `libri100.ipynb` notebook.

In [ ]:
%pip install loralib

As with the `libri100.ipynb` notebook, we need to provide a dictionary to specify the file path and type for each data.

In [ ]:
DUMP_DIR = "./dump/libri100"
data_info = {
    "speech": ["wav.scp", "sound"],
    "text": ["text", "text"],
}

### Load a pretrained model

In ESPnet-Easy, you have the flexibility to define a custom model using the `build_model_fn` method. Additionally, you can load a pretrained model when needed.

In [ ]:
from espnet2.bin.asr_inference import Speech2Text
from espnet2.layers.create_lora_adapter import create_lora_adapter


def build_model_fn(args):
    pretrained_model = Speech2Text.from_pretrained('pyf98/librispeech_conformer_hop_length160')
    model = pretrained_model.asr_model
    model.train()

    # apply lora
    create_lora_adapter(model, target_modules=['linear_q'])
    return model

When working with a pretrained model, the configuration is inherited from the model by default. To activate the LoRA model, it's essential to set the `use_lora` parameter to `True`. This configuration update can be easily achieved using the `update_finetune_config` method.

In [ ]:
import espnetez as ez


pretrained_model = Speech2Text.from_pretrained('pyf98/librispeech_conformer_hop_length160')
pretrain_config = vars(pretrained_model.asr_train_args)
del pretrained_model

finetune_config = ez.config.update_finetune_config(
	'asr',
	pretrain_config,
	'finetune_with_lora.yaml'
)

### Training

Finally, let's start training.

In [ ]:
EXP_DIR = "exp/finetune"
STATS_DIR = "exp/stats_finetune"

trainer = ez.Trainer(
    task='asr',
    train_config=finetune_config,
    train_dump_dir="dump/libri100/train",
    valid_dump_dir="dump/libri100/dev",
    build_model_fn=build_model_fn, # provide the pre-trained model
    data_info=data_info,
    output_dir=EXP_DIR,
    stats_dir=STATS_DIR,
    ngpu=1
)
trainer.collect_stats()

In [ ]:
trainer.train()